In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [5]:
train_ds = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
train_ds.head(10)

In [6]:
test_ds = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
test_ds.head(10)

In [7]:
monthly_data = train_ds.pivot_table(
    index = ['shop_id','item_id'],
    values = ['item_cnt_day'],
    columns = ['date_block_num'],
    fill_value = 0,
    aggfunc='sum')

In [8]:
monthly_data.head(10)

In [9]:
monthly_data.tail(10)

In [10]:
monthly_data.reset_index(inplace = True)
monthly_data.head()

In [11]:
train_data = monthly_data.drop(columns= ['shop_id','item_id'], level=0)

In [12]:
train_data.head()

In [13]:
train_data.fillna(0,inplace = True)
train_data.head()

In [14]:
x_train = np.expand_dims(train_data.values[:,:-1],axis = 2)
y_train = train_data.values[:,-1:]

In [15]:
test_rows = monthly_data.merge(
    test_ds,
    on = ['item_id','shop_id'],
    how = 'right')

In [16]:
test_rows.head()

In [17]:
x_test = test_rows.drop(test_rows.columns[:5], axis=1).drop('ID', axis=1)

In [18]:
x_test.fillna(0,inplace = True)

In [19]:
x_test.head()

In [20]:
x_test = np.expand_dims(x_test,axis = 2)

In [21]:
print(x_train.shape,y_train.shape,x_test.shape)

In [22]:
model = tf.keras.models.Sequential()    
model.add(LSTM(64, input_shape=(33, 1), return_sequences=False))
model.add(Dense(1))
    
model.compile(
    loss = 'mse',
    optimizer = 'adam', 
    metrics = ['mean_squared_error']        
)

In [23]:
history = model.fit(
    x_train, 
    y_train, 
    epochs=10, 
    batch_size=4096,
    verbose=1, 
    shuffle=True,
    validation_split=0.4)

In [25]:
plt.plot(history.history["loss"], color="r")
plt.plot(history.history["val_loss"], color="g")
plt.legend(["Training", "Validation"])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.title('Evaluation')
plt.show()

In [26]:
test_predict = model.predict(x_test)

In [28]:
submission = pd.DataFrame({'ID':test_ds['ID'],'item_cnt_month':test_predict.ravel()})
submission['item_cnt_month'] = submission['item_cnt_month']
submission.to_csv('submission.csv',index = False)